#### Librerias 

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import SnowballStemmer
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import unicodedata

In [2]:
### Descargar stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### Cargue datos

In [3]:
Df = pd.read_csv("C:/Users/USER/OneDrive - universidadean.edu.co/5. NLP/Repositorio NLP/Actividad 1/reviews_booking.csv")
Df

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_25428\1533673389.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Df = pd.read_csv("C:/Users/USER/OneDrive - universidadean.edu.co/5. NLP/Repositorio NLP/Actividad 1/reviews_booking.csv")


,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
1,1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10,"Ubicación, comodidades y limpieza",NaN,Sí
2,2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10,"El lugar esta muy bien ubicado, el apartamento...",NaN,Sí
3,3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10,Todo estuvo perfecto. La ubicación y la relaci...,NaN,Sí
4,4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10,"Es muy cómodo y acogedor, la atención es excel...",NaN,Sí
...,...,...,...,...,...,...,...,...,...,...,...,...
438208,438208,Estados Unidos,Apartamento Suite,1 noche,septiembre de 2024,En familia,22 de octubre de 2024,Fantástico,"9,0",NaN,NaN,No
438209,438209,Colombia,Apartamento de 1 dormitorio,1 noche,octubre de 2024,Persona que viaja sola,9 de octubre de 2024,Excepcional,10,NaN,NaN,No
438210,438210,México,Apartamento de 1 dormitorio,4 noches,agosto de 2024,Persona que viaja sola,6 de septiembre de 2024,Pésimo,"1,0",No me pareció nada,No obstante que dos días antes de mi llegada a...,Sí
438211,438211,Colombia,Apartamento de 1 dormitorio,2 noches,julio de 2024,En pareja,22 de julio de 2024,Excepcional,10,NaN,NaN,No


In [4]:
Df['Calificación'] = Df['Calificación'].astype(str).str.replace(',', '.').astype(float)
Df = Df.drop("Unnamed: 0", axis=1)
Df

,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10.0,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10.0,"Ubicación, comodidades y limpieza",NaN,Sí
2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10.0,"El lugar esta muy bien ubicado, el apartamento...",NaN,Sí
3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10.0,Todo estuvo perfecto. La ubicación y la relaci...,NaN,Sí
4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10.0,"Es muy cómodo y acogedor, la atención es excel...",NaN,Sí
...,...,...,...,...,...,...,...,...,...,...,...
438208,Estados Unidos,Apartamento Suite,1 noche,septiembre de 2024,En familia,22 de octubre de 2024,Fantástico,9.0,NaN,NaN,No
438209,Colombia,Apartamento de 1 dormitorio,1 noche,octubre de 2024,Persona que viaja sola,9 de octubre de 2024,Excepcional,10.0,NaN,NaN,No
438210,México,Apartamento de 1 dormitorio,4 noches,agosto de 2024,Persona que viaja sola,6 de septiembre de 2024,Pésimo,1.0,No me pareció nada,No obstante que dos días antes de mi llegada a...,Sí
438211,Colombia,Apartamento de 1 dormitorio,2 noches,julio de 2024,En pareja,22 de julio de 2024,Excepcional,10.0,NaN,NaN,No


In [5]:
Df.describe()

,Calificación
count,438213.000000
mean,8.401774
std,1.974704
min,1.000000
25%,8.000000
50%,9.000000
75%,10.000000
max,10.000000


### Preprocesamiento

#### 1. Normalización

- Minisculas
- Espacio despues de caracteres
- Eliminar caracteres
- Eliminar numeros
- Eliminar tildes

In [6]:
Df_preprocesado = Df.copy()

# Convertir a minúsculas las columnas específicas
Df_preprocesado["Cosas Positivas"] = Df_preprocesado["Cosas Positivas"].str.lower()
Df_preprocesado["Cosas Negativas"] = Df_preprocesado["Cosas Negativas"].str.lower()

# Añadir espacio después de ciertos caracteres
caracteres = r'[.,;]'
Df_preprocesado["Cosas Positivas"] = Df_preprocesado["Cosas Positivas"].str.replace(caracteres, lambda m: m.group(0) + ' ', regex=True)
Df_preprocesado["Cosas Negativas"] = Df_preprocesado["Cosas Negativas"].str.replace(caracteres, lambda m: m.group(0) + ' ', regex=True)

# Eliminar caracteres especiales
Df_preprocesado["Cosas Positivas"] = Df_preprocesado["Cosas Positivas"].str.replace(r"[\W_]+", " ", regex=True)
Df_preprocesado["Cosas Negativas"] = Df_preprocesado["Cosas Negativas"].str.replace(r"[\W_]+", " ", regex=True)

# Eliminar números
Df_preprocesado["Cosas Positivas"] = Df_preprocesado["Cosas Positivas"].str.replace(r"\d+", "", regex=True)
Df_preprocesado["Cosas Negativas"] = Df_preprocesado["Cosas Negativas"].str.replace(r"\d+", "", regex=True)

# Función para eliminar tildes
def eliminar_tildes(texto):
    if isinstance(texto, str):  # Verifica que sea texto
        return ''.join(
            char for char in unicodedata.normalize('NFD', texto)
            if unicodedata.category(char) != 'Mn'
        )
    return texto

# Aplicar la función a las columnas
Df_preprocesado["Cosas Positivas"] = Df_preprocesado["Cosas Positivas"].apply(eliminar_tildes)
Df_preprocesado["Cosas Negativas"] = Df_preprocesado["Cosas Negativas"].apply(eliminar_tildes)

Df_preprocesado

,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10.0,fue la mejor opcion que pudimos haber tomado e...,NaN,Sí
1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10.0,ubicacion comodidades y limpieza,NaN,Sí
2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10.0,el lugar esta muy bien ubicado el apartamento ...,NaN,Sí
3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10.0,todo estuvo perfecto la ubicacion y la relacio...,NaN,Sí
4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10.0,es muy comodo y acogedor la atencion es excele...,NaN,Sí
...,...,...,...,...,...,...,...,...,...,...,...
438208,Estados Unidos,Apartamento Suite,1 noche,septiembre de 2024,En familia,22 de octubre de 2024,Fantástico,9.0,NaN,NaN,No
438209,Colombia,Apartamento de 1 dormitorio,1 noche,octubre de 2024,Persona que viaja sola,9 de octubre de 2024,Excepcional,10.0,NaN,NaN,No
438210,México,Apartamento de 1 dormitorio,4 noches,agosto de 2024,Persona que viaja sola,6 de septiembre de 2024,Pésimo,1.0,no me parecio nada,no obstante que dos dias antes de mi llegada a...,Sí
438211,Colombia,Apartamento de 1 dormitorio,2 noches,julio de 2024,En pareja,22 de julio de 2024,Excepcional,10.0,NaN,NaN,No


##### Filtrar base positivos

- Se filtran las comentarios con al menos 7 stopwords para garantizar que sean del idioma español
- Se filtran los comentarios con una califcación mayor a 9
- Se filtran comentarios con una logintud minimo da ## para poder obtener mayor informacion util

In [7]:
# Crear una lista de stopwords
stop_words = set(stopwords.words('spanish'))

#Función para contar stopwords en un texto
def count_stopwords(text):
    if not isinstance(text, str):
        return 0
    words = text.split()
    return sum(1 for word in words if word.lower() in stop_words)

# Contar stopwords en la columna 'Cosas Positivas'
stopword_counts = Df_preprocesado['Cosas Positivas'].apply(count_stopwords)

# Filtrar filas con menos de 5 stopwords
Df_filtrado_positivo = Df_preprocesado[stopword_counts >= 7]

# Filtrar filas donde 'Cosas Positivas' tenga más de #### palabra
Df_filtrado_positivo = Df_filtrado_positivo[Df_filtrado_positivo['Cosas Positivas'].str.split().str.len() > 30]

# Filtrar filas donde 'Calificación' sea mayor a 9
Df_filtrado_positivo = Df_filtrado_positivo[Df_filtrado_positivo['Calificación'] > 9]

Df_filtrado_positivo


,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10.0,fue la mejor opcion que pudimos haber tomado e...,NaN,Sí
7,Colombia,NaN,11 noches,julio de 2023,Persona que viaja sola,21 de julio de 2023,Excelente lugar!,10.0,la limpieza todo tal cual la foto pero lo mejo...,todo estuvo muy bien recomendado al,Sí
17,Ecuador,NaN,5 noches,octubre de 2024,Persona que viaja sola,29 de octubre de 2024,Excepcional,10.0,me gusto poder comprar los alimentos a mi gust...,NaN,Sí
52,Colombia,NaN,1 noche,febrero de 2024,En familia,7 de abril de 2024,Excepcional,10.0,ubicado cerca a la embajada alemana para cuand...,NaN,Sí
70,Colombia,NaN,1 noche,agosto de 2022,En familia,9 de agosto de 2022,"Maravilloso alojamiento, recomendado!",10.0,la calidad de personas es maravilloso desde qu...,NaN,Sí
...,...,...,...,...,...,...,...,...,...,...,...
438081,Colombia,Habitación Individual,1 noche,junio de 2024,Persona que viaja sola,2 de junio de 2024,Excelente y buen precio,10.0,el mejor hotel boutique en el que me he quedad...,todo estuvo muy bien,Sí
438083,Colombia,Habitación Individual,1 noche,junio de 2024,Persona que viaja sola,2 de junio de 2024,Confortable y Conveniente,10.0,el personal fue amable y atento la habitacion ...,NaN,Sí
438105,Brasil,Apartamento de 3 dormitorios,5 noches,septiembre de 2023,En familia,10 de septiembre de 2023,"Impecável, excelente",10.0,gostei simplesmente de tudo o apartamento e no...,o unico ponto negativo e a proximidade ao aero...,Sí
438170,Colombia,Apartamento de 1 dormitorio,11 noches,julio de 2024,En familia,5 de agosto de 2024,Excelente,10.0,excelente ubicacion con muchos museos y zonas ...,NaN,Sí


##### Filtrar base negativos

- Se filtran las comentarios con al menos 7 stopwords para garantizar que sean del idioma español
- Se filtran los comentarios con una califcación menor a 7
- Se filtran comentarios con una logintud minimo de 40 para poder obtener mayor informacion util

In [8]:
# Crear una lista de stopwords
stop_words = set(stopwords.words('spanish'))

#Función para contar stopwords en un texto
def count_stopwords(text):
    if not isinstance(text, str):
        return 0
    words = text.split()
    return sum(1 for word in words if word.lower() in stop_words)

# Contar stopwords en la columna 'Cosas Positivas'
stopword_counts = Df_preprocesado['Cosas Negativas'].apply(count_stopwords)

# Filtrar filas con menos de 5 stopwords
Df_filtrado_negativo = Df_preprocesado[stopword_counts >= 7]

# Filtrar filas donde 'Cosas Positivas' tenga más de #### palabra
Df_filtrado_negativo = Df_filtrado_negativo[Df_filtrado_negativo['Cosas Negativas'].str.split().str.len() > 40]

# Filtrar filas donde 'Calificación' sea menor
Df_filtrado_negativo = Df_filtrado_negativo[Df_filtrado_negativo['Calificación'] < 7]

Df_filtrado_negativo


,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
51,Colombia,NaN,1 noche,abril de 2024,En grupo,14 de abril de 2024,Desilusionada,1.0,solo me gusto que estuviera muy cerca de la em...,yo reserve una habitacion y cuando llegamos no...,Sí
53,Colombia,NaN,6 noches,enero de 2024,Persona que viaja sola,20 de enero de 2024,Mal,3.0,la compresion y solucion con el problema de la...,realmente las instalaciones estan en mal estad...,Sí
61,Puerto Rico,NaN,2 noches,marzo de 2023,Persona que viaja sola,2 de abril de 2023,Aceptable,5.0,la ubicacion los espacios fuera de la habitaci...,la habitacion las sabanas en la cama estaban s...,Sí
83,Colombia,NaN,3 noches,octubre de 2022,En grupo,17 de octubre de 2022,Publicidad engañosa en cuanto al precio.,1.0,esta ubicado en una zona segura,doy una calificacion baja ya que al realizar l...,Sí
86,Colombia,NaN,1 noche,octubre de 2022,En pareja,3 de octubre de 2022,Mal servicio,3.0,NaN,el servicio en el hostal es pesimo no hay vaso...,Sí
...,...,...,...,...,...,...,...,...,...,...,...
438073,Italia,Estudio Familiar,3 noches,julio de 2024,En pareja,1 de agosto de 2024,Mal,3.0,lo unico bueno fue la posicion lastima el serv...,la habitacion se veia bastante sucia y la cama...,Sí
438113,Colombia,Habitación Doble,1 noche,mayo de 2024,Persona que viaja sola,20 de mayo de 2024,No volveré ni lo recomendaré a mis amigos o fa...,2.0,que esta experiencia me enseno que no debo cam...,publicidad enganosa ofrecen servicios que no t...,Sí
438157,Chile,Habitación Doble,1 noche,abril de 2024,Persona que viaja sola,25 de abril de 2024,Malo el alojamiento,3.0,NaN,tv sin control remoto banos con pisos muy peli...,Sí
438199,Colombia,Apartamento de 1 dormitorio,8 noches,diciembre de 2023,Persona que viaja sola,26 de diciembre de 2023,Agradable,6.0,creo que el ambiente es bastante agradable aun...,el problema que tuve con un duvet blanco que s...,Sí


#### 2. Tokenizacion


##### Comentarios positivos

In [9]:
# Crear el tokenizador con expresión regular para palabras
tokenizer = RegexpTokenizer(r'\w+')

# Aplicar el tokenizador a cada fila de la columna "Cosas Negativas"
Df_filtrado_positivo["Cosas Positivas Tokenizadas"] = Df_filtrado_positivo["Cosas Positivas"].apply(lambda x: tokenizer.tokenize(x))

# Mostrar el DataFrame con la nueva columna tokenizada
Df_filtrado_positivo["Cosas Positivas Tokenizadas"]

0         [fue, la, mejor, opcion, que, pudimos, haber, ...
7         [la, limpieza, todo, tal, cual, la, foto, pero...
17        [me, gusto, poder, comprar, los, alimentos, a,...
52        [ubicado, cerca, a, la, embajada, alemana, par...
70        [la, calidad, de, personas, es, maravilloso, d...
                                ...                        
438081    [el, mejor, hotel, boutique, en, el, que, me, ...
438083    [el, personal, fue, amable, y, atento, la, hab...
438105    [gostei, simplesmente, de, tudo, o, apartament...
438170    [excelente, ubicacion, con, muchos, museos, y,...
438178    [ubicacion, genial, la, zona, muy, segura, dic...
Name: Cosas Positivas Tokenizadas, Length: 7203, dtype: object

##### Comentarios negativos

In [10]:
# Asegurarse de que los datos de texto estén en formato de string antes de tokenizar
#Df_filtrado_negativo["Cosas Negativas"] = Df_filtrado_negativo["Cosas Negativas"].astype(str)

# Aplicar el tokenizador de palabras a cada fila de la columna "Cosas Negativas"
#Df_filtrado_negativo["Cosas Negativas Tokenizadas"] = Df_filtrado_negativo["Cosas Negativas"].apply(lambda x: word_tokenize(x))

# Mostrar el DataFrame con la nueva columna tokenizada
#Df_filtrado_negativo["Cosas Negativas Tokenizadas"]

In [11]:
# Crear el tokenizador con expresión regular para palabras
tokenizer = RegexpTokenizer(r'\w+')

# Aplicar el tokenizador a cada fila de la columna "Cosas Negativas"
Df_filtrado_negativo["Cosas Negativas Tokenizadas"] = Df_filtrado_negativo["Cosas Negativas"].apply(lambda x: tokenizer.tokenize(x))

# Mostrar el DataFrame con la nueva columna tokenizada
Df_filtrado_negativo["Cosas Negativas Tokenizadas"]

51        [yo, reserve, una, habitacion, y, cuando, lleg...
53        [realmente, las, instalaciones, estan, en, mal...
61        [la, habitacion, las, sabanas, en, la, cama, e...
83        [doy, una, calificacion, baja, ya, que, al, re...
86        [el, servicio, en, el, hostal, es, pesimo, no,...
                                ...                        
438073    [la, habitacion, se, veia, bastante, sucia, y,...
438113    [publicidad, enganosa, ofrecen, servicios, que...
438157    [tv, sin, control, remoto, banos, con, pisos, ...
438199    [el, problema, que, tuve, con, un, duvet, blan...
438210    [no, obstante, que, dos, dias, antes, de, mi, ...
Name: Cosas Negativas Tokenizadas, Length: 8829, dtype: object

#### 3. Eliminar stopwords

##### Comentarios positivos

In [12]:
# Carga las stopwords en español
stop_words = set(stopwords.words('spanish'))
print(f"Stopwords en español: {stop_words}")

def eliminar_stopwords(lista_palabras):
    if isinstance(lista_palabras, list):  # Verifica si es una lista
        procesadas = [palabra.lower() for palabra in lista_palabras if palabra.lower() not in stop_words]
        return procesadas
    else:
        print(f"No es una lista: {lista_palabras}")  # Debug
        return lista_palabras  # Si no es una lista, devuelve sin cambios

Df_filtrado_positivo["Cosas Positivas Sin Stopwords"] = Df_filtrado_positivo["Cosas Positivas Tokenizadas"].apply(eliminar_stopwords)
Df_filtrado_positivo["Cosas Positivas Sin Stopwords"]

Stopwords en español: {'hubiera', 'sentida', 'lo', 'estaríamos', 'tuviéramos', 'seríais', 'hubieras', 'teniendo', 'estuviste', 'una', 'vuestro', 'estas', 'sin', 'del', 'estando', 'tuvieras', 'le', 'tuviese', 'estaréis', 'tus', 'mis', 'sobre', 'nuestras', 'tengan', 'ellos', 'tuvieses', 'hubieran', 'tuvieron', 'nosotras', 'y', 'seas', 'están', 'estoy', 'muy', 'estuvimos', 'tienen', 'has', 'cuando', 'estuviesen', 'quienes', 'tengamos', 'o', 'tened', 'fueras', 'estuve', 'hubisteis', 'estés', 'estados', 'suyo', 'estad', 'estás', 'seáis', 'contra', 'estarías', 'mía', 'de', 'tenías', 'vosotros', 'su', 'otro', 'estuviese', 'hayas', 'sentidos', 'fuéramos', 'más', 'estaría', 'te', 'esas', 'estuviéramos', 'serías', 'habíais', 'para', 'estuvieron', 'haya', 'uno', 'habríais', 'tuyas', 'estos', 'como', 'habré', 'qué', 'habéis', 'desde', 'habría', 'nuestra', 'habrías', 'es', 'tendréis', 'estuviésemos', 'estuvieras', 'sean', 'tuve', 'sí', 'con', 'a', 'tendrán', 'estuvisteis', 'mío', 'estaremos', 'quie

0         [mejor, opcion, pudimos, haber, tomado, bogota...
7         [limpieza, tal, foto, mejor, anfitrion, super,...
17        [gusto, poder, comprar, alimentos, gusto, cuen...
52        [ubicado, cerca, embajada, alemana, cita, buen...
70        [calidad, personas, maravilloso, reciben, dan,...
                                ...                        
438081    [mejor, hotel, boutique, quedado, bogota, aten...
438083    [personal, amable, atento, habitacion, impecab...
438105    [gostei, simplesmente, tudo, apartamento, novo...
438170    [excelente, ubicacion, museos, zonas, turistic...
438178    [ubicacion, genial, zona, segura, dicha, agua,...
Name: Cosas Positivas Sin Stopwords, Length: 7203, dtype: object

##### Comentarios negativos

In [13]:
# Carga las stopwords en español
stop_words = set(stopwords.words('spanish'))
print(f"Stopwords en español: {stop_words}")

def eliminar_stopwords(lista_palabras):
    if isinstance(lista_palabras, list):  # Verifica si es una lista
        procesadas = [palabra.lower() for palabra in lista_palabras if palabra.lower() not in stop_words]
        return procesadas
    else:
        print(f"No es una lista: {lista_palabras}")  # Debug
        return lista_palabras  # Si no es una lista, devuelve sin cambios

Df_filtrado_negativo["Cosas Negativas Sin Stopwords"] = Df_filtrado_negativo["Cosas Negativas Tokenizadas"].apply(eliminar_stopwords)
Df_filtrado_negativo["Cosas Negativas Sin Stopwords"]


Stopwords en español: {'hubiera', 'sentida', 'lo', 'estaríamos', 'tuviéramos', 'seríais', 'hubieras', 'teniendo', 'estuviste', 'una', 'vuestro', 'estas', 'sin', 'del', 'estando', 'tuvieras', 'le', 'tuviese', 'estaréis', 'tus', 'mis', 'sobre', 'nuestras', 'tengan', 'ellos', 'tuvieses', 'hubieran', 'tuvieron', 'nosotras', 'y', 'seas', 'están', 'estoy', 'muy', 'estuvimos', 'tienen', 'has', 'cuando', 'estuviesen', 'quienes', 'tengamos', 'o', 'tened', 'fueras', 'estuve', 'hubisteis', 'estés', 'estados', 'suyo', 'estad', 'estás', 'seáis', 'contra', 'estarías', 'mía', 'de', 'tenías', 'vosotros', 'su', 'otro', 'estuviese', 'hayas', 'sentidos', 'fuéramos', 'más', 'estaría', 'te', 'esas', 'estuviéramos', 'serías', 'habíais', 'para', 'estuvieron', 'haya', 'uno', 'habríais', 'tuyas', 'estos', 'como', 'habré', 'qué', 'habéis', 'desde', 'habría', 'nuestra', 'habrías', 'es', 'tendréis', 'estuviésemos', 'estuvieras', 'sean', 'tuve', 'sí', 'con', 'a', 'tendrán', 'estuvisteis', 'mío', 'estaremos', 'quie

51        [reserve, habitacion, llegamos, dieron, sucia,...
53        [realmente, instalaciones, estan, mal, dias, c...
61        [habitacion, sabanas, cama, sucias, boronas, c...
83        [doy, calificacion, baja, realizar, reserva, a...
86        [servicio, hostal, pesimo, vasos, copas, hielo...
                                ...                        
438073    [habitacion, veia, bastante, sucia, cama, pequ...
438113    [publicidad, enganosa, ofrecen, servicios, cap...
438157    [tv, control, remoto, banos, pisos, peligrosos...
438199    [problema, duvet, blanco, mancho, bocado, choc...
438210    [obstante, dos, dias, llegada, avise, llegaria...
Name: Cosas Negativas Sin Stopwords, Length: 8829, dtype: object

#### 4. Lematización

##### Comentarios positivos

In [14]:
nlp = spacy.load("es_core_news_sm")

# Función para lematizar (sin eliminar stopwords)
def solo_lematizar(lista_palabras):
    if isinstance(lista_palabras, list):  # Asegúrate de que sea una lista
        texto = " ".join(lista_palabras)  # Convierte la lista a un texto
        doc = nlp(texto)  # Procesa el texto con spaCy
        # Lematiza y devuelve todas las palabras (incluyendo stopwords y puntuación si están presentes)
        return [token.lemma_ for token in doc]
    else:
        return lista_palabras  # Devuelve sin cambios si no es una lista

# Aplica la función a la columna
Df_filtrado_positivo["Cosas Positivas Lematizadas"] = Df_filtrado_positivo["Cosas Positivas Sin Stopwords"].apply(solo_lematizar)

# Verifica los resultados
Df_filtrado_positivo["Cosas Positivas Lematizadas"]


0         [mejor, opcion, pudimos, haber, tomar, bogota,...
7         [limpiezar, tal, foto, mejor, anfitrion, super...
17        [gusto, poder, comprar, alimento, gusto, conta...
52        [ubicado, cerca, embajada, alemán, citar, buen...
70        [calidad, persona, maravilloso, recibir, dar, ...
                                ...                        
438081    [mejor, hotel, boutique, quedado, bogota, aten...
438083    [personal, amable, atento, habitacion, impecab...
438105    [gostei, simplesmente, tudo, apartamento, novo...
438170    [excelente, ubicacion, museo, zona, turistica,...
438178    [ubicacion, genial, zona, seguro, dicho, agua,...
Name: Cosas Positivas Lematizadas, Length: 7203, dtype: object

##### Comentarios negativos

In [15]:

nlp = spacy.load("es_core_news_sm")

# Función para lematizar (sin eliminar stopwords)
def solo_lematizar(lista_palabras):
    if isinstance(lista_palabras, list):  # Asegúrate de que sea una lista
        texto = " ".join(lista_palabras)  # Convierte la lista a un texto
        doc = nlp(texto)  # Procesa el texto con spaCy
        # Lematiza y devuelve todas las palabras (incluyendo stopwords y puntuación si están presentes)
        return [token.lemma_ for token in doc]
    else:
        return lista_palabras  # Devuelve sin cambios si no es una lista

# Aplica la función a la columna
Df_filtrado_negativo["Cosas Negativas Lematizadas"] = Df_filtrado_negativo["Cosas Negativas Sin Stopwords"].apply(solo_lematizar)

# Verifica los resultados
Df_filtrado_negativo["Cosas Negativas Lematizadas"]



51        [reserve, habitacion, llegar, dar, sucio, noch...
53        [realmente, instalación, estar, mal, dia, comp...
61        [habitacion, sabana, cama, sucio, borona, cabe...
83        [dar, calificacion, bajar, realizar, reserva, ...
86        [servicio, hostal, pesimo, vaso, copa, hielo, ...
                                ...                        
438073    [habitacion, veiar, bastante, sucio, cama, peq...
438113    [publicidad, enganós, ofrecer, servicio, capac...
438157    [tv, control, remoto, bano, piso, peligroso, a...
438199    [problema, duvet, blanco, mancho, bocado, choc...
438210    [obstante, dos, dia, llegado, avise, llegario,...
Name: Cosas Negativas Lematizadas, Length: 8829, dtype: object

#### 5. Representación numerica

In [16]:
# Inicializa el vectorizador TF-IDF
tfidf_vectorizer= TfidfVectorizer(
    ngram_range=(1, 2),  # Incluye unigramas y bigramas
    min_df=0.02,  # Ajusta para términos más frecuentes (puedes ajustar según tu corpus)
    max_df=0.85   # Excluye términos extremadamente comunes
)

##### Comentarios positivos

In [17]:
Df_filtrado_positivo["Cosas Positivas Sin Stopwords"] = Df_filtrado_positivo["Cosas Positivas Sin Stopwords"].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else x
)

# Ajusta y transforma los datos
tfidf_matrix = tfidf_vectorizer.fit_transform(Df_filtrado_positivo["Cosas Positivas Sin Stopwords"])

# Obtén las características (palabras únicas o bigramas)
tfidf_features = tfidf_vectorizer.get_feature_names_out()

# Convierte la matriz dispersa a un DataFrame
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_features)

# Calcula la importancia promedio de cada término
word_importance = df_tfidf.mean().sort_values(ascending=False)

# Muestra las 20 palabras/bigramas más importantes
top_words_positivas = word_importance.head(40)
print(top_words_positivas)

hotel              0.068172
personal           0.057369
excelente          0.057301
desayuno           0.050454
super              0.048251
ubicacion          0.047869
habitacion         0.045363
atencion           0.044801
cerca              0.039539
bien               0.038821
lugar              0.038471
amable             0.036237
buena              0.036001
bogota             0.034034
servicio           0.033867
siempre            0.031202
zona               0.030113
habitaciones       0.028229
aeropuerto         0.026822
comoda             0.026101
et                 0.025486
mas                0.024818
instalaciones      0.024529
limpio             0.023532
si                 0.023436
mejor              0.022616
buen               0.022319
restaurantes       0.021998
comodas            0.021549
in                 0.021471
gusto              0.021314
cama               0.021199
atento             0.021034
amables            0.020983
precio             0.020784
recepcion          0

##### Comentarios negativos

In [18]:
Df_filtrado_negativo["Cosas Negativas Sin Stopwords"] = Df_filtrado_negativo["Cosas Negativas Sin Stopwords"].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else x
)


# Ajusta y transforma los datos
tfidf_matrix = tfidf_vectorizer.fit_transform(Df_filtrado_negativo["Cosas Negativas Sin Stopwords"])

# Obtén las características (palabras únicas o bigramas)
tfidf_features = tfidf_vectorizer.get_feature_names_out()

# Convierte la matriz dispersa a un DataFrame
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_features)

# Calcula la importancia promedio de cada término
word_importance = df_tfidf.mean().sort_values(ascending=False)

# Muestra las 20 palabras/bigramas más importantes
top_words_negativas = word_importance.head(40)
print(top_words_negativas)



habitacion      0.072173
hotel           0.063671
habia           0.038929
mas             0.036685
desayuno        0.036243
si              0.035527
agua            0.033837
noche           0.031954
bano            0.031837
tenia           0.030573
servicio        0.030386
solo            0.029766
personal        0.029610
habitaciones    0.029389
recepcion       0.028868
dia             0.028094
ruido           0.028013
reserva         0.026042
cama            0.025804
mal             0.024330
the             0.022674
dos             0.021929
in              0.020884
booking         0.020444
atencion        0.020253
precio          0.019462
nunca           0.019234
bien            0.019036
personas        0.018536
toallas         0.017919
dieron          0.017699
lugar           0.017654
mala            0.017595
dias            0.017423
persona         0.016990
hora            0.016972
puerta          0.016405
ducha           0.016370
dormir          0.016327
check           0.016314


#### Preguntas

1. No funciona wordtokenize
2. Se eliminan las tildes?


#### Pendientes

Crear cluster de kmens palabras mas importantes
